In [1]:
import os
import cv2 as cv
import numpy as np
import tensorflow as tf
import tensorflow.keras.backend as K
from matplotlib import pyplot as plt
from tensorflow.keras import activations,layers, models, losses, optimizers, datasets, utils

In [2]:
from functools import reduce
def compose(*funcs):
    """Compose arbitrarily many functions, evaluated left to right.

    Reference: https://mathieularose.com/function-composition-in-python/
    """
    if funcs:
        return reduce(lambda f, g: lambda *a, **kw: g(f(*a, **kw)), funcs)
    else:
        raise ValueError('Composition of empty sequence not supported.')

In [3]:
def custom_relu(x):
    return K.relu(x, max_value=6.0, alpha=0, threshold=0)

def mobileBlock(
    dw_kernel_size, filters, 
    name=None,padding="valid", 
    strides=1, activation=custom_relu):
    
    layers_list=[
        layers.DepthwiseConv2D(
        dw_kernel_size, strides=strides,padding=padding,use_bias=False),
        layers.BatchNormalization(),
        layers.Activation(activation=custom_relu),
        layers.Conv2D(filters,1, padding="same",use_bias=False),
        layers.BatchNormalization(),
        layers.Activation(activation=custom_relu),
    ]
    return compose(*layers_list)

In [4]:
origin_model=tf.keras.applications.MobileNet(
    input_shape=(224,224,3))

2024-04-17 10:26:38.950861: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-04-17 10:26:38.951113: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-04-17 10:26:38.956853: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-04-17 10:26:38.957094: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-04-17 10:26:38.957284: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from S

In [5]:
inp=layers.Input(shape=(224,224,3))
# head
x=compose(
    layers.Conv2D(32,3,strides=2,padding="same",use_bias= False),
    layers.BatchNormalization(),
    layers.Activation(activations.relu),# tf.nn.relu6
)(inp)

x=mobileBlock(3, 64, padding="same",name="block1")(x)
x=layers.ZeroPadding2D(padding=((0,1),(0,1)))(x)

x=mobileBlock(3, 128, strides=2,padding="valid",name="block2")(x)
x=mobileBlock(3, 128, padding="same",name="block3")(x)
x=layers.ZeroPadding2D(padding=((0,1),(0,1)))(x)

x=mobileBlock(3, 256, strides=2,padding="valid",name="block4")(x)
x=mobileBlock(3, 256, padding="same",name="block5")(x)
x=layers.ZeroPadding2D(padding=((0,1),(0,1)))(x)

x=mobileBlock(3, 512, strides=2,padding="valid",name="block6")(x)
x=mobileBlock(3, 512, padding="same",name="block7")(x)
x=mobileBlock(3, 512, padding="same",name="block8")(x)
x=mobileBlock(3, 512, padding="same",name="block9")(x)
x=mobileBlock(3, 512, padding="same",name="block10")(x)
x=mobileBlock(3, 512, padding="same",name="block11")(x)
x=layers.ZeroPadding2D(padding=((0,1),(0,1)))(x)

x=mobileBlock(3, 1024, strides=2,padding="valid",name="block12")(x)
x=mobileBlock(3, 1024, padding="same",name="block13")(x)

x=layers.GlobalAveragePooling2D(keepdims=True)(x)
x=layers.Dropout(rate=0.001)(x)
x=layers.Conv2D(1000,1,padding="same")(x)
x=layers.Reshape((1000,))(x)
out=layers.Activation(activations.softmax)(x)
copy_model=models.Model(inp,out)

In [6]:
copy_model.set_weights(origin_model.get_weights())

for org, new in zip(origin_model.layers, copy_model.layers):
    new._name=org.name